# 🪙 Camada Silver - Enriquecimento e Otimização de Vendas

## 📌 Objetivo

Este pipeline tem como finalidade transformar os dados brutos processados da **Camada Bronze** em uma **tabela desnormalizada, otimizada e confiável** (Esse modelo **One Big Table (OBT)** facilita análises exploratórias, reduz a necessidade de joins complexos e melhora a performance de consultas analíticas) na **Camada Silver**, pronta para análise e consumo por dashboards, relatórios e modelo analítico. Ele aplica práticas avançadas de engenharia de dados com o **Delta Lake** no ambiente **Databricks**. 

---

## 📁 Estrutura de Pastas

| Camada     | Localização |
|------------|-------------|
| Bronze     | `abfss://bronze@dlsprojetofixo.dfs.core.windows.net` |
| Silver     | `abfss://silver@dlsprojetofixo.dfs.core.windows.net` |

---

## 🔍 Fontes Utilizadas

- `vendas_lucas_exploded` — Tabela com informações detalhadas de vendas (por item).
- `clientes_lucas` — Tabela de dados cadastrais dos clientes.

---

## 🛠️ Principais Transformações

- **Conversão de tipos:** `OrderDate` convertida para `Date`
- **Remoção de colunas ambíguas:** `Date_Time_Load` removido de clientes
- **Limpeza de dados:** remoção de duplicatas e nulos (`dropDuplicates()` + `na.drop()`)
- **Desnormalização:** join entre vendas e clientes via chave `CustomerID`
- **Padronização:** ordenação explícita das colunas
- **Inclusão de auditoria:** campo `last_updated` com timestamp da carga
- **Reparticionamento por data:** otimização por `OrderDate`

---

## 🧠 Técnicas Avançadas Utilizadas

| Técnica                    | Benefício                                                                 |
|---------------------------|--------------------------------------------------------------------------|
| 🔄 `DeltaTable.merge()`    | Garantia de atualização incremental e controle de duplicidade           |
| 📦 `broadcast join`       | Acelera joins assimétricos (cliente x vendas) evitando shuffle           |
| 🧹 `dropDuplicates()` + `na.drop()` | Eleva a qualidade do dado removendo inconsistências              |
| 🧊 `cache()`               | Melhora performance em pipelines com múltiplas etapas                    |
| 🧱 `OPTIMIZE ZORDER`       | Melhora leitura por colunas com filtragem frequente (`OrderDate`, `OrderID`) |
| 🧼 `VACUUM`                | Reduz custo de armazenamento com limpeza de arquivos obsoletos           |
| 📚 Registro no metastore  | Permite acesso à tabela Silver via SQL e notebooks                       |

---

## 🧪 Validação Final

Ao final da execução, é feita a contagem total de registros na tabela para fins de auditoria.





| **Informações**    |            **Detalhes**         |
|--------------------|---------------------------------|
| Nome da Tabela     |          Camada Silver          |
| Data da Ingestao   |            31/03/2025           |
| Ultima Atualizaçao |            31/03/2025           |
| Origem             | Banco de Dados Azure - Patrick  |
| Responsável        |           Lucas Sousa           |
| Motivo             |   Criaçao de Camada Silver      |
| Observações        |               None              |

In [0]:
# 📦 Pipeline Silver - Enriquecimento de Vendas com Dados de Clientes
# 🧠 Objetivo: Criar uma tabela desnormalizada e otimizada na camada Silver
# com práticas avançadas de engenharia de dados:
# - Controle de schema
# - Performance em joins
# - Economia de recursos
# - Otimizações estruturais (Z-ORDER, VACUUM)
# - Clareza e rastreabilidade para uso analítico

from pyspark.sql.functions import col, current_timestamp, to_date, broadcast
from delta.tables import DeltaTable

# ------------------------------------------
# ✅ Caminhos das tabelas Delta
# ------------------------------------------

# Fontes da camada Bronze (origem de dados raw processados)
file_path_sales = 'abfss://bronze@dlsprojetofixo.dfs.core.windows.net/vendas_lucas_exploded'
file_path_customer = 'abfss://bronze@dlsprojetofixo.dfs.core.windows.net/clientes_lucas'
# Destino da camada Silver (dados refinados, desnormalizados e preparados para análise)
table_path_silver = 'abfss://silver@dlsprojetofixo.dfs.core.windows.net/tabela_prata_desnormalizadas'

# ------------------------------------------
# ✅ Leitura dos dados da Bronze
# ------------------------------------------
# Leitura no formato Delta permite transações ACID, controle de versão e escalabilidade
vendas_df = spark.read.format("delta").load(file_path_sales)
clientes_df = spark.read.format("delta").load(file_path_customer)

# Remove a coluna ambígua que existe em ambos os DataFrames (evita erro no join)
clientes_df = clientes_df.drop("Date_Time_Load")

# Converte a coluna "OrderDate" para tipo Date (removendo horas)
vendas_df = vendas_df.withColumn("OrderDate", to_date(col("OrderDate")))

# ------------------------------------------
# ✅ Aplicação de cache para evitar reprocessamento dos dados durante o pipeline
# ------------------------------------------
# Ideal para melhorar performance quando os DataFrames são usados múltiplas vezes
vendas_df.cache()
clientes_df.cache()

# ------------------------------------------
# ✅ Join entre Vendas e Clientes usando broadcast join
# ------------------------------------------
# Broadcast envia o DataFrame menor (clientes) para todos os nós do cluster,
# evitando shuffle e melhorando performance drasticamente em joins assimétricos
joined_df = vendas_df.alias("sales").join(
    broadcast(clientes_df.alias("customer")),
    on="CustomerID",
    how="right"
)

# ------------------------------------------
# ✅ Limpeza dos dados
# ------------------------------------------
# Remove registros duplicados e com valores nulos — garante qualidade e consistência
joined_df = joined_df.dropDuplicates()
joined_df = joined_df.na.drop()

# ------------------------------------------
# ✅ Seleção das colunas relevantes e inclusão de auditoria
# ------------------------------------------
# Inclui informações necessárias para análise e auditoria
final_df = joined_df.select(
    col("sales.OrderDate"),
    col("sales.OrderID"),
    col("sales.CustomerID"),
    col("sales.Status"),
    col("sales.ItemID"),
    col("sales.ProductName"),
    col("sales.Quantity"),
    col("sales.Price"),
    col("sales.TotalAmount"),
    col("sales.City"),
    col("sales.Country"),
    col("sales.PostalCode"),
    col("sales.State"),
    col("sales.Street"),
    col("customer.Name"),
    col("customer.PhoneNumber"),
    col("customer.Age").cast("int"),
    current_timestamp().alias("last_updated")  # marca o horário da carga na Silver
)

# ------------------------------------------
# ✅ Reorganização explícita da ordem das colunas para consistência
# ------------------------------------------
# Garante que a tabela Delta Silver tenha layout padronizado, importante para downstream (dashboards, relatórios)
final_df = final_df.select(
    "OrderDate", "OrderID", "CustomerID", "Status", "ItemID", "ProductName",
    "Quantity", "Price", "TotalAmount", "City", "Country", "PostalCode",
    "State", "Street", "Name", "PhoneNumber", "Age", "last_updated"
)

# ------------------------------------------
# ✅ Reparticionamento por OrderDate
# ------------------------------------------
# Estratégia que melhora performance na gravação e futuras leituras filtradas por data
final_df = final_df.repartition(8, "OrderDate")

# ------------------------------------------
# ✅ Criação do banco Silver no catálogo, se necessário
# ------------------------------------------
spark.sql("CREATE DATABASE IF NOT EXISTS silver")

# ------------------------------------------
# ✅ Escrita com MERGE (upsert): atualiza ou insere com base em chaves de negócio
# ------------------------------------------
# Mantém integridade da Silver e evita duplicidade com eficiência
if DeltaTable.isDeltaTable(spark, table_path_silver):
    silver_table = DeltaTable.forPath(spark, table_path_silver)
    silver_table.alias("target").merge(
        final_df.alias("source"),
        """
        target.OrderID = source.OrderID AND 
        target.ItemID = source.ItemID AND 
        target.OrderDate = source.OrderDate
        """
    ).whenMatchedUpdate(set={
        "OrderDate": col("source.OrderDate"),
        "OrderID": col("source.OrderID"),
        "CustomerID": col("source.CustomerID"),
        "Status": col("source.Status"),
        "ItemID": col("source.ItemID"),
        "ProductName": col("source.ProductName"),
        "Quantity": col("source.Quantity"),
        "Price": col("source.Price"),
        "TotalAmount": col("source.TotalAmount"),
        "City": col("source.City"),
        "Country": col("source.Country"),
        "PostalCode": col("source.PostalCode"),
        "State": col("source.State"),
        "Street": col("source.Street"),
        "Name": col("source.Name"),
        "PhoneNumber": col("source.PhoneNumber"),
        "Age": col("source.Age"),
        "last_updated": col("source.last_updated")
    }).whenNotMatchedInsert(values={
        "OrderDate": col("source.OrderDate"),
        "OrderID": col("source.OrderID"),
        "CustomerID": col("source.CustomerID"),
        "Status": col("source.Status"),
        "ItemID": col("source.ItemID"),
        "ProductName": col("source.ProductName"),
        "Quantity": col("source.Quantity"),
        "Price": col("source.Price"),
        "TotalAmount": col("source.TotalAmount"),
        "City": col("source.City"),
        "Country": col("source.Country"),
        "PostalCode": col("source.PostalCode"),
        "State": col("source.State"),
        "Street": col("source.Street"),
        "Name": col("source.Name"),
        "PhoneNumber": col("source.PhoneNumber"),
        "Age": col("source.Age"),
        "last_updated": col("source.last_updated")
    }).execute()
    print("✅ Tabela Silver atualizada com sucesso via MERGE.")
else:
    final_df.write.format("delta").mode("append").save(table_path_silver)
    print("✅ Tabela Silver criada com sucesso.")


# ------------------------------------------
# ✅ Otimização estrutural com Z-ORDER
# ------------------------------------------
# Organiza os dados fisicamente por colunas de acesso frequente (filtragens, joins)
# Melhora performance de leitura filtrada por `OrderDate` e `OrderID`
spark.sql(f"""
    OPTIMIZE delta.`{table_path_silver}`
    ZORDER BY (OrderDate, OrderID)
""")

# ------------------------------------------
# ✅ Limpeza de arquivos antigos com VACUUM
# ------------------------------------------
# Reduz custo de armazenamento, mantendo apenas arquivos úteis
spark.sql(f"""
    VACUUM delta.`{table_path_silver}` RETAIN 168 HOURS
""")

# ------------------------------------------
# ✅ Registro da tabela no catálogo Hive/Unity Catalog
# ------------------------------------------
# Garante disponibilidade por SQL e notebooks
spark.sql(f"""
    CREATE TABLE IF NOT EXISTS silver.tabela_prata_desnormalizadas
    USING DELTA
    LOCATION '{table_path_silver}'
""")

# ------------------------------------------
# ✅ Validação final: contagem de registros
# ------------------------------------------
# Verificação rápida e útil para logs, auditoria e monitoramento
total_registros = spark.read.format("delta").load(table_path_silver).count()
print(f"✅ Total de registros na Tabela Silver: {total_registros}")


✅ Tabela Silver atualizada com sucesso via MERGE.
✅ Total de registros na Tabela Silver: 29945


In [0]:
%sql
SELECT * FROM silver.tabela_prata_desnormalizadas LIMIT 20

CustomerID,OrderDate,OrderID,Status,TotalAmount,ItemID,Price,ProductName,Quantity,City,Country,PostalCode,State,Street,Name,PhoneNumber,Age,last_updated,lastupdate
8,2024-02-04,9761,Pending,682.6,1d8cc57e-147b-4c5e-9eaf-b0359dca6914,315.58,Scanner,1,Jacksonview,Malawi,16236,WI,32501 Larry Parkways,Marie Moon,+1-917-7575,41,2025-04-22T01:39:49.945Z,null
35,2023-02-03,5481,Completed,319.27,8de76938-7ae6-4be1-81a1-2f2fefb3f240,279.71,Motherboard,3,Victorport,Brunei Darussalam,69412,KS,087 Michele Causeway,Sandra Craig,+1-369-8136,66,2025-04-22T01:39:49.945Z,null
78,2024-04-18,6372,Pending,582.61,8bd3bf24-1f80-4045-b73d-3023f7783596,325.22,Power Supply,3,East Scottburgh,Fiji,18353,WY,3453 Peter Wells,Dustin Foster,+1-623-7342,29,2025-04-22T01:39:49.945Z,null
159,2022-12-11,569,Pending,46.12,4f568819-4a3f-4bc4-9797-7e49ab345b06,476.18,Smartwatch,1,Garciaside,Singapore,95226,KS,48677 Elizabeth Prairie,Jerry Butler,+1-459-1917,34,2025-04-22T01:39:49.945Z,null
170,2023-04-13,1542,Cancelled,629.66,506aa919-581e-43cc-8beb-8b7aeb7ec06a,48.13,USB Cable,1,West Lisaborough,Canada,49641,NC,190 Farley Avenue Apt. 470,Edward Miller,+1-858-7541,35,2025-04-22T01:39:49.945Z,null
204,2024-10-17,7886,Completed,614.27,057077d9-a384-4ff5-9ff3-45c91fe6c92b,446.18,USB Cable,2,Port Lauramouth,Saint Lucia,23477,KS,0242 Frank Walks Suite 511,Robert Burns,+1-662-5893,64,2025-04-22T01:39:49.945Z,null
210,2023-01-09,4042,Pending,355.37,22c01a68-6480-4848-a284-beabcde1f167,160.59,Power Supply,1,Francesfort,Falkland Islands (Malvinas),12328,ID,996 Gonzalez Turnpike Suite 293,Nicole Butler,+1-569-8131,37,2025-04-22T01:39:49.945Z,null
215,2024-01-08,8511,Completed,720.51,53b3e58a-bad8-4985-8deb-96398ac1b4ba,224.41,Smartwatch,3,New Johnview,Brunei Darussalam,92728,WI,193 Rebecca Drives,Paul Lopez,+1-630-9754,38,2025-04-22T01:39:49.945Z,null
264,2022-12-24,1569,Pending,243.08,8af1473e-30ff-4c71-abfc-fca2aa32f097,292.0,Tablet,3,West Ericaland,Bolivia,29957,MS,904 Fowler Cove,Diane Stewart,+1-887-2644,46,2025-04-22T01:39:49.945Z,null
289,2024-04-11,1876,Cancelled,904.26,ef0392af-9dc9-48db-92a1-d969149d43c7,54.36,Gaming Chair,2,North Bianca,Cote d'Ivoire,31997,ME,871 Karla Valley,Sarah George,+1-273-2739,60,2025-04-22T01:39:49.945Z,null


In [0]:
# 📊 Monitoramento e Governança - Log de Execução do Pipeline Silver

from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType
from pyspark.sql.functions import current_timestamp
from datetime import datetime
import time

# ------------------------------------------
# ✅ Início da contagem do tempo de execução
# ------------------------------------------
start_time = time.time()

# ------------------------------------------
# ✅ Parâmetros do log
# ------------------------------------------
job_name = "silver_tabela_prata_desnormalizadas"
status = "SUCESSO"
erro = None

try:
    qtd_linhas = total_registros  # vindo da última célula

except Exception as e:
    status = "ERRO"
    erro = str(e)
    qtd_linhas = 0

# ------------------------------------------
# ✅ Tempo total de execução em segundos
# ------------------------------------------
tempo_total = round(time.time() - start_time, 2)

# ------------------------------------------
# ✅ Esquema explícito para evitar erros de inferência
# ------------------------------------------
schema_log = StructType([
    StructField("job_name", StringType(), True),
    StructField("data_execucao", StringType(), True),
    StructField("qtd_linhas", IntegerType(), True),
    StructField("status", StringType(), True),
    StructField("erro", StringType(), True),
    StructField("tempo_total_segundos", DoubleType(), True)
])

# ------------------------------------------
# ✅ Criação do DataFrame de log
# ------------------------------------------
log_execucao_df = spark.createDataFrame([(
    job_name,
    datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    qtd_linhas,
    status,
    erro,
    tempo_total
)], schema=schema_log)

# ------------------------------------------
# ✅ Escrita no Delta Lake - agora dentro do container 'silver'
# ------------------------------------------
log_execucao_df.write.format("delta") \
    .mode("append") \
    .option("mergeSchema", "true") \
    .save("abfss://silver@dlsprojetofixo.dfs.core.windows.net/log_execucoes_silver")

print(f"📌 Log da execução do job '{job_name}' registrado com sucesso.")


📌 Log da execução do job 'silver_tabela_prata_desnormalizadas' registrado com sucesso.


In [0]:
%sql
CREATE TABLE IF NOT EXISTS silver.log_execucoes_silver
USING DELTA
LOCATION 'abfss://silver@dlsprojetofixo.dfs.core.windows.net/log_execucoes_silver'

In [0]:
%sql
SELECT * FROM silver.log_execucoes_silver 

job_name,data_execucao,qtd_linhas,status,erro,tempo_total_segundos
silver_tabela_prata_desnormalizadas,2025-04-22 02:06:34,29945,SUCESSO,null,0.0
